In [28]:
import pandas as pd
import sklearn
import lightgbm as lgb
from operator import itemgetter
import optuna
import numpy as np
from sklearn.decomposition import *

In [69]:
train_data = pd.read_csv('1031andHSIC_train.csv')
train_data.drop('Unnamed: 0',axis=1,inplace=True)
test_data = pd.read_csv('1031andHSIC_test.csv')
test_data.drop('Unnamed: 0',axis=1,inplace=True)

In [1]:
import os
try:
    os.makedirs('predictions')
except OSError:
    pass

In [86]:
param = {
    'objective': 'regression',
    'metric': 'l2',
    'device': 'gpu',
    'tree_learner':'voting',
    'max_bin': 1000,
    'boosting':'dart',
    'n_estimators':2000,
    'early_stopping_rounds':50,
    'learning_rate': 0.1,
    }
y_values = train_data['GA']
X = train_data.drop('GA',axis=1)

dtrain = lgb.Dataset(X, label=y_values)

reg = lgb.train(param, dtrain)
y_preds = reg.predict(test_data)
pd.DataFrame(y_preds).to_csv('predictions/Lightgbm_preds.csv',index=False)

reg = sklearn.linear_model.Ridge(alpha=38)
reg.fit(X,y_values)
y_preds = reg.predict(test_data)
pd.DataFrame(y_preds).to_csv('predictions/Ridge_preds.csv',index=False)
    

In [ ]:
param = {
    'objective': 'regression',
    'metric': 'l2',
    'device': 'gpu',
    'tree_learner':'voting',
    'max_bin': 1000,
#     'max_bin': 2000,
    'boosting':'dart',
#     'boosting':'goss',
#     'boosting':'gbdt',
    'n_estimators':2000,
    'early_stopping_rounds':50,
#         'max_depth':5,
#         'num_leaves':31,
    'learning_rate': 0.1,
#     'learning_rate': 0.075,
#         'l1_lambda':0.7,
#         'l2_lambda':0.5,
}
dtrain = lgb.Dataset(X, label=y)
gbm = lgb.cv(param, dtrain, stratified=False, seed=220, nfold=5,verbose_eval=True)
print(gbm['l2-mean'])